In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import VarianceThreshold
import shap
from yellowbrick.regressor import ResidualsPlot

In [ ]:
import sys  
sys.path.insert(0, '../src')

In [ ]:
from preprocess import *
from artifacts import *

**Attribute Information:**

<span style="color:red ">**Date**</span>: (DD/MM/YYYY)  

<span style="color:red ">**Time**</span>: (HH.MM.SS)   

<span style="color:red ">**CO**</span>: True hourly averaged concentration mg/m^3 (reference analyzer)   

<span style="color:red ">**PT08.S1**</span>: (tin oxide) hourly averaged sensor response (nominally CO targeted   

<span style="color:red ">**NMHC**</span>: True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)  

<span style="color:red ">**C6H6(GT)**</span>: True hourly averaged Benzene concentration in microg/m^3 (reference analyzer) 

<span style="color:red ">**PT08.S2**</span>: (titania) hourly averaged sensor response (nominally NMHC targeted)   

<span style="color:red ">**NOx**</span>: True hourly averaged concentration in ppb (reference analyzer)    

<span style="color:red ">**PT08.S3**</span>: (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)       

<span style="color:red ">**NO2**</span>: True hourly averaged concentration in microg/m^3 (reference analyzer)  

<span style="color:red ">**PT08.S4**</span>: (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted) 

<span style="color:red ">**PT08.S5**</span>: indium oxide) hourly averaged sensor response (nominally O3 targeted) 

<span style="color:red ">**PT08.S5**</span>: (indium oxide) hourly averaged sensor response (nominally O3 targeted)     

<span style="color:red ">**Temperature**</span>: Temperature in Â°C  

<span style="color:red ">**Relative Humidity**</span>: (%)  

<span style="color:red ">**Absolute Humidity**</span>: AH  



In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('../data/AirQualityUCI/AirQualityUCI.csv',index_col=0)

# feature engineering

In [ ]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' '+ df['Time'])
df['day_name'] = df['DateTime'].apply(lambda x: x.day_name())
df['month_name'] = df['DateTime'].apply(lambda x: x.month_name())
df['time_of_day'] = df['DateTime'].apply(find_time_of_day)

# imputation

In [ ]:
df= df.applymap(lambda x: np.nan if x==-200 else x)

In [ ]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

In [ ]:
df.drop(columns=['NMHC(GT)','DateTime','Date','Time'],inplace=True)

In [ ]:
non_chemical_features = ['AH','RH','T']
chemical_features = ['CO(GT)', 'C6H6(GT)','NO2(GT)','NOx(GT)']
sensor_chemical_features = [ 'PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)', 'PT08.S1(CO)']

In [ ]:
sns.pairplot(df[sensor_chemical_features+chemical_features],diag_kind='hist')

### normalize features

In [ ]:
df_normalized = df.copy()
for col in chemical_features + sensor_chemical_features + non_chemical_features:
    df_normalized[col]=(df_normalized[col]-df_normalized[col].min())/(df_normalized[col].max()-df_normalized[col].min())

In [ ]:
df_normalized.sample(4)

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0,add_indicator=True)

In [ ]:
column_names = sensor_chemical_features+chemical_features

In [ ]:
missing_names = ['missing_'+x for x in column_names]

In [ ]:
df_sensor_chemical_imputed = pd.DataFrame(imp.fit_transform(df_normalized[sensor_chemical_features+chemical_features]),
                                         columns=column_names + missing_names)

In [ ]:
df_sensor_chemical_imputed['missing'] = (df_sensor_chemical_imputed[missing_names].sum(axis=1)!=0).astype(int)

In [ ]:
# sns.pairplot(df_sensor_chemical_imputed[column_names+['missing']],hue='missing')

In [ ]:
df_sensor_chemical_imputed[chemical_features+['missing']]

In [ ]:
### impute temperature and humidity features

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0,add_indicator=True)

In [ ]:
missing_names = ['missing_'+x for x in non_chemical_features]

In [ ]:
df_normalized.sample(3)

In [ ]:
df_temp_hum= pd.DataFrame(imp.fit_transform(df_normalized[non_chemical_features]),
                                         columns=non_chemical_features + missing_names)

In [ ]:
df_temp_hum['missing'] = (df_temp_hum[missing_names].sum(axis=1)!=0).astype(int)

In [ ]:
df_temp_hum.sample(3)

In [ ]:
df_sensor_chemical_imputed.sample(3)

In [ ]:
df_final = df_sensor_chemical_imputed[chemical_features+['missing']]

In [ ]:
df_final.loc[:,'missing'] = df_final.loc[:,'missing'] +df_temp_hum.loc[:,'missing']


In [ ]:
df_final['missing'] = df_final['missing'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
df_final[non_chemical_features] = df_temp_hum[non_chemical_features]

In [ ]:
df_final.loc[:,'total_chemicals'] = df_final.loc[:,chemical_features].sum(axis=1)

In [ ]:
df_final.drop(columns=chemical_features,inplace=True)

### dummify categorical variables

In [ ]:
numerical_columns = list(df._get_numeric_data().columns)
cat_columns = [x for x in df.columns if x not in numerical_columns]

In [ ]:
df_dummies = pd.get_dummies(df,columns = cat_columns)

In [ ]:
df_dummies.drop(columns=numerical_columns,inplace=True)

In [ ]:
df_final = pd.concat([df_dummies,df_final],axis=1)

### remove outliers

In [ ]:
outlier_columns = ['AH','RH','T','total_chemicals']
from scipy import stats

In [ ]:
df_final_trimmed= df_final[(np.abs(stats.zscore(df_final[outlier_columns])) < 3).all(axis=1)]

In [ ]:
# fig = plt.figure(facecolor='white')
# sns_plot = sns.pairplot(df_final_trimmed[non_chemical_features + ['total_chemicals','missing']],
#                         hue='missing',kind='hist',corner=True)
# plt.tight_layout()
# sns_plot.savefig("../plots/imputed_pairplot.png",facecolor=fig.get_facecolor(), edgecolor='none',dpi=100)

### build model

In [ ]:
targets = ['total_chemicals']
features = [x for x in df_final.columns if (x not in targets and 'missing' not in x)]

In [ ]:
X = df_final_trimmed[features]
y = df_final_trimmed[targets]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
param_grid = {'n_estimators':[10,50,100],
              'max_depth':[2,10,50,100,300,600],
              'max_features':[1,2,4,9,20,30],
             'verbose':[1]}

In [ ]:
search = GridSearchCV(estimator=RandomForestRegressor(),param_grid=param_grid, cv = 5)

In [ ]:
search.fit(X_train,y_train)

In [ ]:
r2_score(y_true=y_train,y_pred=search.predict(X_train))

In [ ]:
r2_score(y_true=y_test,y_pred=search.predict(X_test))

In [ ]:
param_grid_evaluation(model=search,param_grid=param_grid,metric='mean_test_score')

In [ ]:
fig = plt.figure()
visualizer = ResidualsPlot(search,hist=False, qqplot=True,show=False)
visualizer.fit(np.array(X_train), np.array(y_train.squeeze(1)))  # Fit the training data to the visualizer
visualizer.score(np.array(X_test), np.array(y_test.squeeze(1)))  # Evaluate the model on the test data
visualizer.finalize()
plt.tight_layout()
plt.savefig('../plots/residual_analysis.png')

# feature importance

In [ ]:
visualize_n_important_features(df=df_final_trimmed[features+targets],n=5,model=search,target=['total_chemicals'])

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model=search.best_estimator_)

In [ ]:
shap_values = explainer.shap_values(X_train)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
# summarize the effects of all the features
fig = plt.figure(facecolor='white')
shap_plot = shap.summary_plot(shap_values, X_train,show=False)
plt.tight_layout()
plt.savefig('../plots/shap.png',facecolor=fig.get_facecolor(), edgecolor='none',dpi=100)

In [ ]:
shap.dependence_plot("time_of_day_sleep_time", shap_values, X_train)

In [ ]:
stop here